In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"soumikkaran","key":"64bedf532f0253a786f278b6e7e17ddb"}'}

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d rmisra/news-category-dataset
!unzip news-category-dataset.zip


Dataset URL: https://www.kaggle.com/datasets/rmisra/news-category-dataset
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/26.5M [00:00<?, ?B/s]
100% 26.5M/26.5M [00:00<00:00, 878MB/s]
Archive:  news-category-dataset.zip
  inflating: News_Category_Dataset_v3.json  


In [6]:
!unzip news-category-dataset.zip


Archive:  news-category-dataset.zip
replace News_Category_Dataset_v3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
error:  invalid response [{ENTER}]
replace News_Category_Dataset_v3.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
!unzip -o news-category-dataset.zip


Archive:  news-category-dataset.zip
  inflating: News_Category_Dataset_v3.json  


In [8]:
import pandas as pd
import json

data = []
with open("News_Category_Dataset_v3.json", "r") as file:
    for line in file:
        data.append(json.loads(line))

df = pd.DataFrame(data)
df = df[['headline', 'category']]  # Keep only text and label
df.head()


,headline,category
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY
3,The Funniest Tweets From Parents This Week (Se...,PARENTING
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS


In [9]:
from sklearn.preprocessing import LabelEncoder

# Encode text labels into numbers
label_encoder = LabelEncoder()
df['category_encoded'] = label_encoder.fit_transform(df['category'])

# Check unique categories and their encoded values
print(dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))


{'ARTS': np.int64(0), 'ARTS & CULTURE': np.int64(1), 'BLACK VOICES': np.int64(2), 'BUSINESS': np.int64(3), 'COLLEGE': np.int64(4), 'COMEDY': np.int64(5), 'CRIME': np.int64(6), 'CULTURE & ARTS': np.int64(7), 'DIVORCE': np.int64(8), 'EDUCATION': np.int64(9), 'ENTERTAINMENT': np.int64(10), 'ENVIRONMENT': np.int64(11), 'FIFTY': np.int64(12), 'FOOD & DRINK': np.int64(13), 'GOOD NEWS': np.int64(14), 'GREEN': np.int64(15), 'HEALTHY LIVING': np.int64(16), 'HOME & LIVING': np.int64(17), 'IMPACT': np.int64(18), 'LATINO VOICES': np.int64(19), 'MEDIA': np.int64(20), 'MONEY': np.int64(21), 'PARENTING': np.int64(22), 'PARENTS': np.int64(23), 'POLITICS': np.int64(24), 'QUEER VOICES': np.int64(25), 'RELIGION': np.int64(26), 'SCIENCE': np.int64(27), 'SPORTS': np.int64(28), 'STYLE': np.int64(29), 'STYLE & BEAUTY': np.int64(30), 'TASTE': np.int64(31), 'TECH': np.int64(32), 'THE WORLDPOST': np.int64(33), 'TRAVEL': np.int64(34), 'U.S. NEWS': np.int64(35), 'WEDDINGS': np.int64(36), 'WEIRD NEWS': np.int64(37

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters
vocab_size = 10000  # Number of words to keep
max_length = 50     # Max words in a headline
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"

# Create tokenizer and fit on headlines
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(df['headline'])

# Convert headlines to sequences of numbers
sequences = tokenizer.texts_to_sequences(df['headline'])

# Pad sequences so they all have same length
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)


In [11]:
from sklearn.model_selection import train_test_split

X = padded
y = df['category_encoded'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training samples: {len(X_train)}")
print(f"Testing samples: {len(X_test)}")


Training samples: 167621
Testing samples: 41906


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

num_classes = len(label_encoder.classes_)

model = Sequential([
    Embedding(vocab_size, 64, input_length=max_length),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')  # Softmax for multi-class classification
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X_train, y_train, epochs=5, validation_data=(X_test, y_test))


Epoch 1/5
5239/5239 ━━━━━━━━━━━━━━━━━━━━ 351s 66ms/step - accuracy: 0.1733 - loss: 3.2869 - val_accuracy: 0.2410 - val_loss: 2.8863
Epoch 2/5
 646/5239 ━━━━━━━━━━━━━━━━━━━━ 5:49 76ms/step - accuracy: 0.2448 - loss: 2.8702